In [9]:
import pandas as pd

In [10]:
# load unbalanced pool of all viable sentences (not downsampled)
d=pd.read_parquet('../sentence_pool/data/output/final_sentence_pool_unbalanced.parquet')

In [11]:
# create subsets with only specific party and for left also with speciifc source
left = d[(d['source_party']=='Left') | (d['source_party']=='Lean Left')]
right = d[(d['source_party']=='Right') | (d['source_party']=='Lean Right')]
center = d[d['source_party']=='Center']
alternet = d[d['source_name']=='alternet']


In [12]:
# read the annomatic dev/test to prevent data leakage
dev = pd.read_parquet('../annomatic-dataset/data/training/anno-lexical-dev.parquet')
test = pd.read_parquet('../annomatic-dataset/data/training/anno-lexical-test.parquet')
dev_test = pd.concat([dev,test])

In [13]:
# remove sentences from dataframes that are in the annolexical dev/test set
left = left[~left['text'].isin(dev_test['text'])]
right = right[~right['text'].isin(dev_test['text'])]
center = center[~center['text'].isin(dev_test['text'])]
alternet = alternet[~alternet['text'].isin(dev_test['text'])]

In [15]:
# read annotations to prevent redundant annotations
annotated=pd.read_parquet('../annomatic-dataset/data/output/final_sentence_pool_annotated.parquet')
annotated=annotated[['text','label_zephyr','label_openchat','label_llama','final_label']]

In [16]:
# set the already annotated sentences

right = pd.merge(right, annotated, on='text', how='left')
left = pd.merge(left, annotated, on='text', how='left')
center = pd.merge(center, annotated, on='text', how='left')
alternet = pd.merge(alternet, annotated, on='text', how='left')

In [77]:
# split into annotated and yet to be annotated and save

right_annotated = right[~right.final_label.isna()]
right2annotate = right[right.final_label.isna()]

right_annotated.to_parquet('data/right/right_train_annotated.parquet')
right2annotate[['text','source_party','source_name']].to_parquet('data/right/right_train_to_annotate.parquet')

In [78]:
left_annotated = left[~left.final_label.isna()]
left2annotate = left[left.final_label.isna()]

left2annotate_b = left2annotate[left2annotate.bias_estimate==1].sample(30000)
left2annotate_n = left2annotate[left2annotate.bias_estimate==0].sample(len(left2annotate_b))

left2annotate = pd.concat([left2annotate_b,left2annotate_n]).sample(frac=1)

left_annotated.to_parquet('data/left/left_train_annotated.parquet')
left2annotate[['text','source_party','source_name']].to_parquet('data/left/left_train_to_annotate.parquet')

In [79]:
center_annotated = center[~center.final_label.isna()]
center2annotate = center[center.final_label.isna()]

center_annotated.to_parquet('data/center/center_train_annotated.parquet')
center2annotate[['text','source_party','source_name']].to_parquet('data/center/center_train_to_annotate.parquet')

In [85]:
alternet_annotated = alternet[~alternet.final_label.isna()]
alternet2annotate = alternet[alternet.final_label.isna()]

alternet_annotated.to_parquet('data/alternet/alternet_train_annotated.parquet')
alternet2annotate[['text','source_party','source_name']].to_parquet('data/alternet/alternet_train_to_annotate.parquet')